### Import Packages

In [1]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np

import pandas as pd
import folium
import pulp
from geopy.distance import distance as geodist
import scipy
import itertools
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

import networkx as nx
import osmnx as ox
import geopandas as gpd
from shapely.geometry import LineString, Point
from ast import literal_eval
import json
import random
import math

D:\Anaconda\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [32]:
pics = pd.read_csv('dump_loc_img.csv')

import webbrowser

for i in pics:
    webbrowser.open(pics['streetview'][i])

KeyError: '_key'

### Varanasi City View Using Folium Package

In [2]:
#tiles="cartodbpositron"
#tiles="Stamen Toner"
map = folium.Map(location=[25.322314334637344, 82.99574380727432], zoom_start=13, control_scale=True, 
                 tiles="Stamen Terrain")
map

#### Load Bin Data

In [3]:
bin_data = pd.read_csv('varanasi_swm_bin_updated.csv')
bin_data

FileNotFoundError: [Errno 2] No such file or directory: 'varanasi_swm_bin_updated.csv'

#### Create DataFrame for Bin Locations

In [ ]:
# Create Empty DataFrame for Bin locations
bin_loc = pd.DataFrame(columns = ['latitude','longitude'])
i=0
for row in bin_data['location.coordinates']:
    # Load Latitude and Longitude data in each row of bin_data to a dummy DataFrame (dummy_df)
    # literal_eval() function converts str -> list and np.array() converts list -> array
    dummy_df = pd.DataFrame({'latitude': np.array(literal_eval(row))[1], 'longitude': np.array(literal_eval(row))[0]}, index=[i])
    bin_loc = pd.concat([bin_loc, dummy_df]) # Concatenate DateFrame dummy_df to bin_loc DataFrame
    i+=1

# Print bin_loc DataFrame    
bin_loc

#### Load Dumping Data

In [5]:
# Opening JSON file
dump_data = pd.read_json("dump_loc_img.json")
dump_data

,893c16454a3ffff,893c1645663ffff,893c1645243ffff,893c164524fffff,893c1645637ffff,893c164562bffff,893c1644637ffff,893c16450d3ffff,893c164543bffff,893c16456a7ffff,893c164548bffff,893c164509bffff,893c1647277ffff,893c1647267ffff,893c16454a7ffff,893c16456afffff,893c164546fffff
streetview,https://storiesdata.s3.ap-south-1.amazonaws.co...,https://storiesdata.s3.ap-south-1.amazonaws.co...,https://storiesdata.s3.ap-south-1.amazonaws.co...,https://storiesdata.s3.ap-south-1.amazonaws.co...,https://storiesdata.s3.ap-south-1.amazonaws.co...,https://storiesdata.s3.ap-south-1.amazonaws.co...,https://storiesdata.s3.ap-south-1.amazonaws.co...,https://storiesdata.s3.ap-south-1.amazonaws.co...,https://storiesdata.s3.ap-south-1.amazonaws.co...,https://storiesdata.s3.ap-south-1.amazonaws.co...,https://storiesdata.s3.ap-south-1.amazonaws.co...,https://storiesdata.s3.ap-south-1.amazonaws.co...,https://storiesdata.s3.ap-south-1.amazonaws.co...,https://storiesdata.s3.ap-south-1.amazonaws.co...,https://storiesdata.s3.ap-south-1.amazonaws.co...,https://storiesdata.s3.ap-south-1.amazonaws.co...,https://storiesdata.s3.ap-south-1.amazonaws.co...
location,"[82.992505, 25.349756]","[82.98561816444558, 25.31384543966206]","[82.993738, 25.297353]","[82.99737227617469, 25.29695312499696]","[82.990132, 25.324171]","[82.988633, 25.317318]","[83.02494, 25.373419]","[82.993845, 25.331153]","[82.99532883206489, 25.344196195633174]","[82.987564, 25.325695]","[82.98151971928104, 25.34572039635309]","[82.997632, 25.333361]","[82.973523, 25.351861]","[82.975388, 25.350061]","[82.992478, 25.349794]","[82.987637, 25.32564]","[82.993971, 25.331207]"


#### Create DataFrame for Dumping Locations

In [6]:
# Create Eampty DataFrame for dumping locations
dump_loc = pd.DataFrame(columns = ['latitude','longitude'])
for i in range(dump_data.shape[1]):
    # Load Latitude and Longitude data of each hex_id location to a dummy DataFrame (dummy_df)
    dummy_df = pd.DataFrame({'latitude': dump_data.loc['location'].values[i][1], 'longitude': dump_data.loc['location'].values[i][0]}, index=[i])
    dump_loc = pd.concat([dump_loc, dummy_df]) # Concatenate DateFrame dummy_df to bin_loc DataFrame

# Add Extra location using DataFrame (NTPC)
dummy_df = pd.DataFrame({'latitude': 25.2206677, 'longitude': 82.9214183}, index=[i])  

# Concatenate New dumping location with dump_loc DataFrame and reset the index
dump_loc = pd.concat([dump_loc, dummy_df])
dump_loc.drop([0,7,9], inplace=True)
dump_loc.reset_index(inplace=True)
dump_loc = dump_loc.drop(['index'], axis=1)

# Print dumping locations DataFrame
dump_loc

,latitude,longitude
0,25.313845,82.985618
1,25.297353,82.993738
2,25.296953,82.997372
3,25.324171,82.990132
4,25.317318,82.988633
5,25.373419,83.024940
6,25.344196,82.995329
7,25.345720,82.981520
8,25.333361,82.997632
9,25.351861,82.973523


#### Load and Create DataFrame for Vehicle Starting Locations

In [7]:
# Locading Starting Location Data into a DataFrame
start_data = pd.read_csv('top_35_starting_hexes.csv')
#start_data = start_data.drop(['Unnamed: 0'], axis=1)
#start_data = start_data.rename(columns = {'0': 'latitude', '1': 'longitude'})

# Sort the DataFrame with 'no_of_points' row and reset the index
start_data = start_data.sort_values('no_of_points').reset_index(inplace=False).drop(['index'], axis=1)

# Remove first 10 values
start_data = start_data.iloc[10:, :].reset_index(inplace=False)

# Print DataFrame
start_data

,index,h3_id_7,no_of_points,h3_center_point
0,10,873c16470ffffff,4,"(25.377289949830857, 82.97417061865464)"
1,11,873c1640bffffff,4,"(25.31238806264238, 82.9578210013321)"
2,12,873c16440ffffff,4,"(25.378804042138928, 83.03885453501607)"
3,13,873c164eeffffff,7,"(25.2706025047615, 83.01157235889963)"
4,14,873c164e0ffffff,8,"(25.27682061056327, 82.97028786721233)"
5,15,873c1640dffffff,12,"(25.347939169433356, 82.94535217934816)"
6,16,873c16473ffffff,12,"(25.363377404836314, 82.99209259003328)"
7,17,873c16455ffffff,14,"(25.349458976945318, 83.0100152165335)"
8,18,873c164ecffffff,18,"(25.292250298829256, 83.0170266811429)"
9,19,873c16442ffffff,20,"(25.357171232841598, 83.03339598639693)"


#### Compute Vehicle Distribution based on number of visits

$$Fraction\ of\ vehicle\ visits = \frac{Number\ of\ visits\ at\ each\ location}{Total\ number\ of\ visits\ at\ all\ locations} \times Total\ number\ of\ all\ available\ vehicles $$
$$ Number\ of\ vehicles\ visiting\ each\ location = round\ to\ nerest\ integer(Fraction\ of\ vehicle\ visits)$$

In [8]:
# Compute Fraaction of Vehicle Visits to all locations and Normalise with number of available vehicles
fraction_of_vehicle_visits = np.rint((start_data['no_of_points'].values/start_data['no_of_points'].values.sum())*150)

# Extract indices of the nonzero start locations (locations with atleast one vehicle visit)
start_indices = np.nonzero(fraction_of_vehicle_visits)

# Extract Vehicles allocations to start locations using start indices
vehicle_allocation = fraction_of_vehicle_visits[start_indices]

# Create New DataFrame for Starting locations and Assign location data
start_locations = start_data.iloc[start_indices[0], :].reset_index(inplace=False)

# Drop Unnecessary Columns
start_locations = start_locations.drop(['level_0','index'], axis=1)

# Print start_locations DataFrame
start_locations

,h3_id_7,no_of_points,h3_center_point
0,873c164e5ffffff,78,"(25.298465418018964, 82.97573746374873)"
1,873c16451ffffff,96,"(25.335534668665368, 83.02793849468361)"
2,873c16472ffffff,120,"(25.3556597343234, 82.96871968994382)"
3,873c16453ffffff,125,"(25.313894355396936, 83.02248205966826)"
4,873c16476ffffff,143,"(25.36956976475606, 82.95080021446434)"
5,873c16454ffffff,152,"(25.34174382094886, 82.98663982495094)"
6,873c16450ffffff,768,"(25.327822027136065, 83.00456061601592)"
7,873c16452ffffff,1676,"(25.30618133218442, 82.99910707193877)"
8,873c1642bffffff,2009,"(25.3618466858841, 82.92743520505378)"
9,873c16446ffffff,2171,"(25.37109217582371, 83.01547087369963)"


#### Create DataFrame for Starting Locations

In [9]:
# Create New DataFrame for starting Locations - start_loc with columns 'latitude','longitude', 'vehicles'
start_loc = pd.DataFrame(columns = ['latitude','longitude', 'vehicles'])
i = 0
for row in start_locations['h3_center_point']:
    # Load Latitude and Longitude data in each row of start_locations to a dummy DataFrame (dummy_df)
    # literal_eval() function converts str -> list and np.array() converts list -> array
    dummy_df = pd.DataFrame({'latitude': literal_eval(row)[0], 'longitude':
                             literal_eval(row)[1],
                            'vehicles': int(vehicle_allocation[i])}, index=[i])
    
    # Concatenate DateFrame dummy_df to bin_loc DataFrame
    start_loc = pd.concat([start_loc, dummy_df])
    i+=1
start_loc

,latitude,longitude,vehicles
0,25.298465,82.975737,1
1,25.335535,83.027938,1
2,25.355660,82.968720,1
3,25.313894,83.022482,1
4,25.369570,82.950800,1
5,25.341744,82.986640,1
6,25.327822,83.004561,7
7,25.306181,82.999107,15
8,25.361847,82.927435,18
9,25.371092,83.015471,20


#### Droping Markers for Dumping, Bins, and Vehicle Starting Locations on Map using Folium

In [10]:
# Place Markers for Dumping Locations using Folium Map
for index, row in dump_loc.iterrows():
    #Setup the content of the popup
    iframe = folium.IFrame('Dump Number: ' + str(index))
    #Initialise the popup using the iframe
    popup = folium.Popup(iframe, min_width=100, max_width=100)
    folium.Marker([row["latitude"], row["longitude"]],popup=popup, icon=folium.Icon(icon='flag',color="green")).add_to(map)

# Place Markers for Bin Locations using Folium Map
for index, row in bin_loc.iterrows():
    #Setup the content of the popup
    iframe = folium.IFrame('Bin Number: ' + str(index))
    #Initialise the popup using the iframe
    popup = folium.Popup(iframe, min_width=100, max_width=100)
    folium.Marker([row["latitude"], row["longitude"]],popup=popup,).add_to(map)   

# Place Markers for Start Locations using Folium Map
for index, row in start_loc.iterrows():
    #Setup the content of the popup
    iframe = folium.IFrame('Parking Number: ' + str(index))
    #Initialise the popup using the iframe
    popup = folium.Popup(iframe, min_width=100, max_width=100)
    folium.Marker([row["latitude"], row["longitude"]],popup=popup, icon=folium.Icon(icon='home',color="gray")).add_to(map)

# Display Map    
map

#### Initializations and Assumptions

In [12]:
# Get Number of Dumping Locations
dump_count = len(dump_loc)

# Get Number of Bin Locations
bin_count = len(bin_loc)

# Get Number of Starting Locations
start_count = len(start_loc)

# Get Number of Available Locations
vehicle_count = start_loc['vehicles'].sum()#int(np.ceil(vehicle_allocation/start_scale).sum())

# Get Bin Occupancies
#Finding it from updated Bin values
#bin_occupancy = np.full((bin_count), 1)
bin_occupancy = bin_data['binCapacity'].to_list()

# Get Vehicle Capacities
#vehicle_capacity = np.full((vehicle_count), np.ceil(bin_occupancy.sum()/vehicle_count)+5)
#vehicle_capacity = np.full((vehicle_count), np.ceil(sum(bin_occupancy)/vehicle_count)+5)
print(bin_occupancy, vehicle_capacity)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 

### Function to Create Location DataFrame

In [14]:
def create_location_data(vehicle_count):
    # Create New DataFrame for all (Dump, Bin, Start) Location Information
    location_info = pd.DataFrame(columns = ['latitude', 'longitude', 'type', 'occupancy'])


    """ Add Dumping Locations Data. Added a Set of all Dumping Locations for Every Vehicle to facilitate each 
    vehicle to visit any of the available Dumping locations (Each node is allowed only one visit from all the 
    available vehicles, hence created duplicate nodes). All availble dumping locations are duplicated by 
    number_of_vehicles times Number_of_dumping_locations (Number_of_dumping_locations * Number of Vehicles) and
    each vehicle is assigned a set of all available Duplicated Dumping locations. """
    for i in range(vehicle_count):    
        for ind, row in dump_loc.iterrows():
            # Concatenate Latitude, Longitude, Type, and Occupancy Data of Dumping locations with location_info DataFrame
            location_info = pd.concat([location_info, pd.DataFrame({'latitude': row['latitude'], 
                                                                    'longitude': row['longitude'],
                                                                    'type': 'Dump'+' '+str(ind), 'occupancy': 0},
                                                                    index= [ind])])

    # Add Starting Locations Data
    for ind, row in start_loc.iterrows():
        # Concatenate Latitude, Longitude, Type, and Occupancy Data of Starting locations with location_info DataFrame
        location_info = pd.concat([location_info, pd.DataFrame({'latitude': row['latitude'], 
                                                                    'longitude': row['longitude'],
                                                                    'type': 'Parking', 'occupancy': 0},
                                                                    index= [ind])])
            

    # Add Bin Locations Data
    for ind, row in bin_loc.iterrows():
        # Concatenate Latitude, Longitude, Type, and Occupancy Data of Starting locations with location_info DataFrame
        location_info = pd.concat([location_info, pd.DataFrame({'latitude': row['latitude'], 
                                                                    'longitude': row['longitude'],
                                                                    'type': 'Bin', 
                                                                    'occupancy': bin_occupancy[ind]},
                                                                    index= [ind + dump_loc.shape[0]])])


    # Get Number of Total Locations
    location_count = location_info.shape[0]

    # Reset DataFrame Index
    location_info = location_info.reset_index().drop('index', axis=1)

    # Return location_info DataFrame
    return location_info

#### Create DataFrame for All the Locations

In [15]:
#location_info = create_location_data(vehicle_count)

# Return location_info DataFrame
location_info = pd.read_csv('Notebooks/Location Info.csv')
location_info

,Unnamed: 0,latitude,longitude,type,occupancy
0,0,25.313845,82.985618,Dump 0,0
1,1,25.297353,82.993738,Dump 1,0
2,2,25.296953,82.997372,Dump 2,0
3,3,25.324171,82.990132,Dump 3,0
4,4,25.317318,82.988633,Dump 4,0
...,...,...,...,...,...
3012,3012,25.281585,82.984330,Bin,1
3013,3013,25.315730,83.004250,Bin,1
3014,3014,25.312030,82.985840,Bin,1
3015,3015,25.278770,82.967300,Bin,1


#### Compute Vehicle Allocations

In [17]:
# Get Assigned Vehicles to Each Starting Location
assigned_vehicles = list(start_loc['vehicles'].values)

# Get Index of all Starting Locations and Convert into list
parking_loc = list(location_info[location_info['type'] == 'Parking'].index)

# Create Empty list for Vehicles starting indices
vehicle_allocation = []
for i in range(len(assigned_vehicles)):
    # Duplicate each starting location into number of assigned vehicles and add this list to created eampty list 
    vehicle_allocation = vehicle_allocation + [parking_loc[i]]*assigned_vehicles[i]

#### Compute Pairwise Distance between All the Locations

In [ ]:
#Don't run this is distance_matrix already exists, run next cell
# Compute Pairwise Distance Between all available Locations
pairwise_distance = scipy.spatial.distance.cdist(location_info[['latitude','longitude']].to_numpy(), 
                                                 location_info[['latitude','longitude']].to_numpy(),
                                                             lambda u, v: geodist(u, v).m*100)
# Convert Distance matrix to type integer
pairwise_distance = pairwise_distance.astype(int)

# Print Distance Matrix
pairwise_distance

#### Save Pairwise Distance Matrix into a text file

In [55]:
# Save Computed Pairwise Distance Matrix into a text file
#np.savetxt("pairwise_distance_meters.txt", pairwise_distance)

#### Load Pairwise Distance Matrix from text file, convert to type 'int', and print the matrix

In [18]:
# Load Pre-computed Pairwise Distance Matrix from text file
#pairwise_distance = np.loadtxt('pairwise_distance_meters.txt')

# Convert Distance matrix to type integers
#pairwise_distance = pairwise_distance.astype(int)

# Print Distance Matrix
#pairwise_distance

pairwise_distance = pd.read_csv('Notebooks/Matrix_IUDX_Final.csv')
pairwise_distance = pairwise_distance.to_numpy()
#from numpy import genfromtxt
#pairwise_distance = genfromtxt('Notebooks/Matrix_IUDX_Final.csv', delimiter=',')
pairwise_distance = pairwise_distance.astype(int)
pairwise_distance

array([[   0,    0, 2880, ...,  364, 6302, 2758],
       [   1, 2872,    0, ..., 2542, 4312, 1031],
       [   2, 3011,  427, ..., 2681, 4474,  603],
       ...,
       [3014,  337, 2568, ...,    0, 5990, 2446],
       [3015, 6477, 4517, ..., 6148,    0, 7184],
       [3016, 2758, 1031, ..., 2429, 6985,    0]])

### Function to Create Data Model

In [19]:
def create_data_model(location_info, vehicle_count, pairwise_distance, vehicle_allocation):
    """ Creates and Manipulates Data for the Problem """
    
    # Create an Empty Dictionary to Store Data
    data = {}
    
    # Assign Location Data to 'locations' key in data dictionary
    data['locations'] = list(zip([*range(len(location_info))], location_info['type'].values))
    
    # Assign High Penality for Travelling Between Dumping and Bin Locations
    pairwise_distance[:vehicle_count*dump_count, vehicle_count*dump_count+start_count:] = 9999999
    
    # Assign High Penality for Travelling Between Bin and Starting Locations
    pairwise_distance[vehicle_count*dump_count+start_count:, vehicle_count*dump_count : vehicle_count*dump_count+start_count] = 9999999
    
    # Convert Pairwise Distance Matrix to list and Assign it to 'distance_matrix' key in data dictionary
    data['distance_matrix'] = pairwise_distance.tolist()
    
    # Get Occupancy data, convert to list, and Assign it to 'demands' key in data dictionary
    data['demands'] = location_info['occupancy'].values.tolist()
    
    # Assign Vehicle Capacities to 'vehicle_capacities' key in data Dictionary
    data['vehicle_capacities'] = [6]*vehicle_count #list(vehicle_capacity.astype(int))
    
    # Assign Number of Vehicles to 'num_vehicles' key in data Dictionary
    data['num_vehicles'] = vehicle_count
    
    # Assign starting locations of vehicles to 'starts' key in data Dictionary
    data['starts'] = vehicle_allocation
    
    # Assign ending locations of vehicles to 'ends' key in data Dictionary
    data['ends'] = vehicle_allocation

    # Return data Dictionary
    return data

### Route Optimization: Solves the CVRP Problem

#### Get Data, Manager, and Routing Objects

In [20]:
def data_manager_routing(location_info, vehicle_count, pairwise_distance, vehicle_allocation):
    
    # Get the Required Data
    data = create_data_model(location_info, vehicle_count, pairwise_distance, vehicle_allocation)
    
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['starts'],
                                           data['ends'])
    
    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)
    
    return data, manager, routing

#### Function Performs Optimization

In [21]:
def run_optimization(data, manager, routing):
    
    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Get list of bin location indices
    bin_list = [manager.NodeToIndex(x[0]) for x in data['locations'] if x[1] == 'Bin']

    # Get list of Dumping locatin indices
    dump_list = list(range(vehicle_count*dump_count))

    # Get list of Starting location indices
    start_list = [manager.NodeToIndex(x[0]) for x in data['locations'] if x[1] == 'Parking']

    # Get list of Ending location indices
    end_list = [routing.End(i) for i in range(manager.GetNumberOfVehicles())]
    #"""
    # Add Constraint that Vehicles from staring locations can only go to Bin and End locations.
    for node, node_type in data['locations']:
        index = manager.NodeToIndex(node)
        if node_type == "Parking":
            routing.NextVar(index).SetValues(bin_list + end_list)
        
    for item in location_info['type'][:dump_count]:
        # For all the available Vehicles
        for i in range(vehicle_count):
            # Get the index range of the current route or vehicles
            dumpi_list = list(range(dump_count*i, dump_count*(i+1)))
            # For all the Dumping Locations
            for j in range(dump_count):
                if item == "Dump"+' '+str(j):
                    # Remove the index already visited
                    dumpi_list.remove(dump_count*i+j)
                    # Delete the remaining nodes assigned to this route or vehicle using AddDisjuction method.
                    routing.AddDisjunction(dumpi_list, 99999)#"""
                
    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(demand_callback_index,
                                        0,  # null capacity slack
                                        data['vehicle_capacities'],  # vehicle maximum capacities
                                        True,  # start cumul to zero
                                        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(1000)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        print('>>>>>> Solution Found <<<<<<<')
    else:
        print('!!!! XXX No Solution Found XXX !!!!!')
    return solution

#### Get Number of Vehicles Used in the Optimization

In [22]:
def get_vehicles_used():
    """ Gives Number of vehicles used in the optimization. """
    
    # Define an empty list for all the vehicles used in the list
    vehicles_list = []
    
    # For loop for all the vehicles
    for vehicle_id in range(data['num_vehicles']):
        
        # Get starting Index of current vehicle
        start_index = routing.Start(vehicle_id)
        # Get next node from stating index or node
        next_node = manager.IndexToNode(start_index)
        # If staring node and immediate next node are equal, collect the vehicle identity
        if start_index != next_node:
            # Update vehicle list
            vehicles_list += [vehicle_allocation[vehicle_id]]

    
    # Return Vehicles Used
    return len(vehicles_list), vehicles_list

### Run Optimization

In [23]:
data, manager, routing = data_manager_routing(location_info, vehicle_count, pairwise_distance, vehicle_allocation)
solution = run_optimization(data, manager, routing)

>>>>>> Solution Found <<<<<<<


### Export and Print Solution

In [24]:
# Create an empty Dictionary for all vehicles
Solution_Dictionary = {}
print(f'Objective: {solution.ObjectiveValue()}')

# Variable for total distance
total_distance = 0
# Variable for total load
total_load = 0

# For all the given vehicles
for vehicle_id in range(data['num_vehicles']):
    # Get index of staring node of current vehicle
    index = routing.Start(vehicle_id)
    # Variable for update and printing the output
    plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
    # Variable for route distance
    route_distance = 0
    # Variable for route load
    route_load = 0
    # Create a name for current vehicle: Format: Vehicle Number
    Name = 'Vehicle'+' '+str(vehicle_id)
    # Create an empty ditionary for current vehicle
    Solution_Dictionary[Name] = {}
    # Create an empty list for current vehicle load
    load = []
    # Create an empty list for current vehicle distance covered
    distance = []
    # Create an empty list for all the nodes visisted current vehicle path
    node_number = []
    # Create an empty list for type of nodes visited current vehicle 
    node_type = []
    
    # While the current index is not the end index
    while not routing.IsEnd(index):
        # Convert current index to node index
        node_index = manager.IndexToNode(index)
        # Get load of the current node index
        route_load += data['demands'][node_index]
        # Assign current index to previous index
        previous_index = index
        # Get next node index of the vehicle
        index = solution.Value(routing.NextVar(index))
        # Get distance between curent and previous nodes
        route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        # Format the output: NodeIndex(NodeType) Load(Value) Distance(Value) -> 
        plan_output += ' {0}({2}) Load({1}) Distance({3}) -> '.format(node_index, route_load, location_info['type'][node_index],routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id))
        # Update load of the current node visited by the current vehicle or route
        load += [data['demands'][node_index]]
        # Update distance from previous node to current node visited by current vehicle or route
        distance += [routing.GetArcCostForVehicle(previous_index, index, vehicle_id)]
        # Update nodes visited by the current vehicle or route
        node_number += [node_index]
        # Update type of nodes visited by the current vehicle or route
        node_type += [location_info['type'][node_index]]
    # Format the output last visited node: NodeIndex(NodeType) Load(Value) Distance(Value) -> 
    plan_output += ' {0}({2}) Load({1}) Distance({3})\n'.format(manager.IndexToNode(index),
                        route_load, location_info['type'][manager.IndexToNode(index)], routing.GetArcCostForVehicle(
                        previous_index, index, vehicle_id))
    # Update load of the last node visited by the current vehicle or route
    load += [data['demands'][manager.IndexToNode(index)]]
    # Update distance from previous node to last node visited by current vehicle or route
    #distance += [routing.GetArcCostForVehicle(previous_index, index, vehicle_id)]
    # Update last node visited by the current vehicle or route
    node_number += [manager.IndexToNode(index)]
    # Update type of last node visited by the current vehicle or route
    node_type += [location_info['type'][manager.IndexToNode(index)]]
    
    # Assign loads to current vehicle dictionary
    Solution_Dictionary[Name]['Load'] = load
    # Assign distance to current vehicle dictionary
    Solution_Dictionary[Name]['Distance'] = distance
    # Assign all nodes to current vehicle dictionary
    Solution_Dictionary[Name]['Node'] = node_number
    # Assign node types visited by to current vehicle dictionary
    Solution_Dictionary[Name]['Type'] = node_type
    
    # Format the output: Add total distance travelled by the current vehicle
    plan_output += 'Distance of the route: {}m\n'.format(route_distance)
    # Format the output: Add total load collected by the current vehicle
    plan_output += 'Load of the route: {}\n'.format(route_load)
    
    # Print Output of the current vehicle
    print(plan_output)
    
    # Total distance travelled by all the vehicles
    total_distance += route_distance
    # Total Load collected by all the vehicles
    total_load += route_load

# Print total distance travelled and total load collected by all the vehicles
print('Total distance of all routes: {}m'.format(total_distance/100))
print('Total load of all routes: {}'.format(total_load))

Objective: 15564117
Route for vehicle 0:
 2190(Parking) Load(0) Distance(1523) ->  2370(Bin) Load(1) Distance(2018) ->  2867(Bin) Load(2) Distance(280) ->  3011(Bin) Load(3) Distance(0) ->  3012(Bin) Load(4) Distance(0) ->  3013(Bin) Load(5) Distance(442) ->  2712(Bin) Load(6) Distance(1512) ->  749(Dump 14) Load(6) Distance(0) ->  2190(Parking) Load(6) Distance(0)
Distance of the route: 5775m
Load of the route: 6

Route for vehicle 1:
 2191(Parking) Load(0) Distance(698) ->  2919(Bin) Load(1) Distance(374) ->  2884(Bin) Load(2) Distance(281) ->  2267(Bin) Load(3) Distance(200) ->  2758(Bin) Load(4) Distance(200) ->  2268(Bin) Load(5) Distance(445) ->  2361(Bin) Load(6) Distance(289) ->  1861(Dump 1) Load(6) Distance(2166) ->  2191(Parking) Load(6) Distance(2166)
Distance of the route: 4653m
Load of the route: 6

Route for vehicle 2:
 2192(Parking) Load(0) Distance(1700) ->  2731(Bin) Load(1) Distance(0) ->  2730(Bin) Load(2) Distance(0) ->  2729(Bin) Load(3) Distance(83) ->  2981(Bin)

In [25]:
Solution_Dictionary

{'Vehicle 0': {'Load': [0, 1, 1, 1, 1, 1, 1, 0, 0],
  'Distance': [1523, 2018, 280, 0, 0, 442, 1512, 0],
  'Node': [2190, 2370, 2867, 3011, 3012, 3013, 2712, 749, 2190],
  'Type': ['Parking',
   'Bin',
   'Bin',
   'Bin',
   'Bin',
   'Bin',
   'Bin',
   'Dump 14',
   'Parking']},
 'Vehicle 1': {'Load': [0, 1, 1, 1, 1, 1, 1, 0, 0],
  'Distance': [698, 374, 281, 200, 200, 445, 289, 2166],
  'Node': [2191, 2919, 2884, 2267, 2758, 2268, 2361, 1861, 2191],
  'Type': ['Parking',
   'Bin',
   'Bin',
   'Bin',
   'Bin',
   'Bin',
   'Bin',
   'Dump 1',
   'Parking']},
 'Vehicle 2': {'Load': [0, 1, 1, 1, 1, 1, 1, 0, 0],
  'Distance': [1700, 0, 0, 83, 51, 656, 494, 4811],
  'Node': [2192, 2731, 2730, 2729, 2981, 2645, 2637, 1063, 2192],
  'Type': ['Parking',
   'Bin',
   'Bin',
   'Bin',
   'Bin',
   'Bin',
   'Bin',
   'Dump 13',
   'Parking']},
 'Vehicle 3': {'Load': [0, 1, 1, 1, 1, 1, 1, 0, 0],
  'Distance': [2518, 178, 371, 1723, 288, 0, 1658, 1203],
  'Node': [2193, 2891, 2364, 2244, 2213,

In [57]:
with open("Solution_Dictionary_MatrixIUDXFinal_updatedBin.json", "w") as outfile:
    json.dump(Solution_Dictionary, outfile, sort_keys=True, indent=4)
    
json_object = json.dumps(Solution_Dictionary, indent = 4) 
print(json_object)

{
    "Vehicle 0": {
        "Load": [
            0,
            1,
            1,
            1,
            1,
            1,
            1,
            0,
            0
        ],
        "Distance": [
            1523,
            2018,
            280,
            0,
            0,
            442,
            1512,
            0
        ],
        "Node": [
            2190,
            2370,
            2867,
            3011,
            3012,
            3013,
            2712,
            749,
            2190
        ],
        "Type": [
            "Parking",
            "Bin",
            "Bin",
            "Bin",
            "Bin",
            "Bin",
            "Bin",
            "Dump 14",
            "Parking"
        ]
    },
    "Vehicle 1": {
        "Load": [
            0,
            1,
            1,
            1,
            1,
            1,
            1,
            0,
            0
        ],
        "Distance": [
            698,
            374,
       

### Visualization of Routes

In [26]:
import networkx as nx
import osmnx as ox
import geopandas as gpd
from shapely.geometry import LineString, Point
import requests
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm 

#### Reference Article

https://towardsdatascience.com/visualization-in-python-finding-routes-between-points-2d97d4881996

#### Get Drive Network form OSM Data

In [27]:
# location where you want to find your route
place     = 'Varanasi, Uttar Pradesh, India'
# find shortest route based on the mode of travel
mode      = 'drive'        # 'drive', 'bike', 'walk'
# find shortest path based on distance or time
optimizer = 'time'        # 'length','time'
# create graph from OSM within the boundaries of some 
# geocodable place(s)
graph = ox.graph_from_place(place, network_type = mode)
# find the nearest node to the start location

In [28]:
colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']

#### Function to get Shortest Route Between Two Points

In [29]:
def get_shortest_route(start_latlng, end_latlng):
    orig_node = ox.distance.nearest_nodes(graph, start_latlng[1], start_latlng[0])
    # find the nearest node to the end location
    dest_node = ox.distance.nearest_nodes(graph, end_latlng[1], end_latlng[0])
    #  find the shortest path
    shortest_route = nx.shortest_path(graph,
                                      orig_node,
                                  dest_node,
                                  weight=optimizer)
    #print(shortest_route[:1])
    #for i in shortest_route[:1]:
    #    point = graph.nodes[i]
    #    print(point)
    #    print(point['x'])
    #    print(point['y'])
    return shortest_route

In [26]:
get_shortest_route(25.322314334637344, 82.99574380727432)

TypeError: 'float' object is not subscriptable

In [30]:
def get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):

    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://router.project-osrm.org/route/v1/driving/"
    r = requests.get(url + loc)
    if r.status_code!= 200:
        return {}
    res = r.json()
    return res
'''
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    distance = res['routes'][0]['distance']

    out = {
           'start_point':start_point,
           'end_point':end_point,
           'distance':distance
          }

    return out
'''

"\n    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]\n    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]\n    distance = res['routes'][0]['distance']\n\n    out = {\n           'start_point':start_point,\n           'end_point':end_point,\n           'distance':distance\n          }\n\n    return out\n"

In [31]:
route_map = folium.Map(location=[25.322314334637344, 82.99574380727432], zoom_start=11, control_scale=True, 
                 tiles="cartodbpositron")
route_map

In [34]:
def plot_route_map(vehicle_number):
    route_name = 'Vehicle'+' '+str(vehicle_number)
    for j in range(len(Solution_Dictionary[route_name]['Node'])-1):
        start_index = Solution_Dictionary[route_name]['Node'][j]
        end_index = Solution_Dictionary[route_name]['Node'][j+1]
        
        if Solution_Dictionary[route_name]['Type'][j] == 'Bin':
            #Setup the content of the popup
            iframe = folium.IFrame(Solution_Dictionary[route_name]['Type'][j])
            #Initialise the popup using the iframe
            popup = folium.Popup(iframe, min_width=100, max_width=100)
            folium.Marker([location_info.iloc[start_index,:2]['latitude'], location_info.iloc[start_index,:3]['longitude']]
                          ,popup=popup,).add_to(route_map)
        elif Solution_Dictionary[route_name]['Type'][j] == 'Parking':
            #Setup the content of the popup
            iframe = folium.IFrame(Solution_Dictionary[route_name]['Type'][j])
            #Initialise the popup using the iframe
            popup = folium.Popup(iframe, min_width=100, max_width=100)
            folium.Marker([location_info.iloc[start_index,:2]['latitude'], location_info.iloc[start_index,:3]['longitude']]
                          ,popup=popup, icon=folium.Icon(icon='home',color="darkpurple")).add_to(route_map)
        else:
            #Setup the content of the popup
            iframe = folium.IFrame(Solution_Dictionary[route_name]['Type'][j])
            #Initialise the popup using the iframe
            popup = folium.Popup(iframe, min_width=100, max_width=100)
            folium.Marker([location_info.iloc[start_index,:2]['latitude'], location_info.iloc[start_index,:3]['longitude']]
                          ,popup=popup, icon=folium.Icon(icon='flag',color="green")).add_to(route_map)
            
        if end_index == len(Solution_Dictionary[route_name]['Node'])-1:
            #Setup the content of the popup
            iframe = folium.IFrame(Solution_Dictionary[route_name]['Type'][j])
            #Initialise the popup using the iframe
            popup = folium.Popup(iframe, min_width=100, max_width=100)
            folium.Marker([location_info.iloc[start_index,:2]['latitude'], location_info.iloc[start_index,:3]['longitude']]
                          ,popup=popup, icon=folium.Icon(icon='home',color="darkpurple")).add_to(route_map)
        
        if Solution_Dictionary[route_name]['Distance'][j] > 100:
            """if j == len(Solution_Dictionary[route_name]['Node']-1):
                end_index = Solution_Dictionary[route_name]['Node'][0]
                shortest_route = get_shortest_route(location_info.iloc[start_index,:2], 
                                                    location_info.iloc[end_index,:2])
                shortest_route_map = ox.plot_route_folium(graph, shortest_route, route_map = route_map,
                color = 'blue')
            else:"""  
            end_index = Solution_Dictionary[route_name]['Node'][j+1]
            try:
                shortest_route = get_shortest_route(location_info.iloc[start_index,:2], 
                                                        location_info.iloc[end_index,:2])
                shortest_route_map = ox.plot_route_folium(graph, shortest_route, route_map = route_map, zoom=1,
                                                          color = random.choice(colors))
            except:
                route_name = 'Vehicle'+' '+str(vehicle_number)

In [37]:
# save map to html file
route_map.save('route_map_MatrixIUDX_updated.html')

In [46]:
for i in range(10):
    route_name = 'Vehicle'+' '+str(i)
    for j in range(len(Solution_Dictionary[route_name]['Node'])-1):
        start_index = Solution_Dictionary[route_name]['Node'][j]
        end_index = Solution_Dictionary[route_name]['Node'][j+1]
        if Solution_Dictionary[route_name]['Distance'][j] != 0:
            if j == len(Solution_Dictionary[route_name]['Node']):
                end_index = Solution_Dictionary[route_name]['Node'][0]
                shortest_route = get_shortest_route(location_info.iloc[start_index,:2], 
                                                    location_info.iloc[end_index,:2])
                shortest_route_map = ox.plot_route_folium(graph, shortest_route, route_map = route_map)
            else:
                end_index = Solution_Dictionary[route_name]['Node'][j+1]
                shortest_route = get_shortest_route(location_info.iloc[start_index,:2], 
                                                    location_info.iloc[end_index,:2])
                shortest_route_map = ox.plot_route_folium(graph, shortest_route, route_map = route_map)

In [ ]:
list(range(5))

In [147]:
vehicles_used, vehicle_allocation_updated = get_vehicles_used()
location_info_updated = create_location_data(vehicles_used)
pairwise_distance_updated = pairwise_distance[(vehicle_count-vehicles_used)*dump_count:, (vehicle_count-vehicles_used)*dump_count:]

In [148]:
data_updated, manager_updated, routing_updated = data_manager_routing(location_info_updated, 
                                            vehicles_used, pairwise_distance_updated, vehicle_allocation_updated)

In [149]:
solution_updated = run_optimization(data_updated, manager_updated, routing_updated)

!!!! XXX No Solution Found XXX !!!!!


In [ ]:
for vehicle_id in range(data['num_vehicles']):
    # Get index of staring node of current vehicle
    index = routing.Start(vehicle_id)
    print(index)
    print(manager.IndexToNode(index))

In [8]:
import folium
latlon = [ (51.249443914705175, -0.13878830247011467), (51.249443914705175, -0.13878830247011467), (51.249768239976866, -2.8610415615063034)]
mapit = folium.Map( location=[52.667989, -1.464582], zoom_start=6 )
for coord in latlon:
    folium.Marker( location=[ coord[0], coord[1] ], fill_color='#43d9de', radius=8 ).add_to( mapit )
    shortest_route = get_shortest_route(location_info.iloc[start_index,:2], 
                                                    location_info.iloc[end_index,:2])
            shortest_route_map = ox.plot_route_folium(graph, shortest_route, route_map = route_map, zoom=1,
                                                      color = colors[vehicle_number])

In [9]:
mapit